<a href="https://colab.research.google.com/github/saisuresh1999/Assignment/blob/master/assignment_4_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

Install and import all the necessary libraries for the assignment.

In [89]:
!pip install tensorflow==2.0.0-rc0

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column
from tensorflow import keras
from tensorflow.keras import layers



tf.random.set_seed(1)

### **Importing the dataset**

In [0]:
boston_dataset = load_boston()

data_X = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
data_Y = pd.DataFrame(boston_dataset.target, columns=["target"])
data = pd.concat([data_X, data_Y], axis=1)

In [91]:
train, test = train_test_split(data, test_size=0.2, random_state=1)
train, val = train_test_split(train, test_size=0.2, random_state=1)
print(len(train), "train examples")
print(len(val), "validation examples")
print(len(test), "test examples")

323 train examples
81 validation examples
102 test examples


Converting the Pandas DataFrames into Tensorflow Datasets

In [0]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 32
train_ds = df_to_dataset(train, True, batch_size)
val_ds = df_to_dataset(val, False, batch_size)
test_ds = df_to_dataset(test, False, batch_size)

### Defining Feature Columns

In [0]:
# define feature_columns as a list of features using functions from tf.feature_column

feature_columns = []
for header in ['CRIM'	,'ZN',	'INDUS',	'CHAS'	,'NOX'	,'RM'	,'AGE'	,'DIS'		,'TAX'	,'PTRATIO',	'B',	'LSTAT']:
  feature_columns.append(feature_column.numeric_column(header))
rad = feature_column.numeric_column("RAD")
rad_buckets = feature_column.bucketized_column(rad, boundaries= [2, 5] )
feature_columns.append(rad_buckets)


### Building the model

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

Model should contain following layers:

```
feature_layer

Dense(1, activation=None)
```

Use 'Adam' optimizer

Use 'mse' as your loss and metric

In [0]:
# Build and compile your model in this cell.
 model = keras.Sequential([
    feature_layer,
     layers.Dense(1,activation=None)
     
  ])

model.compile(loss='mse',
                optimizer='adam',
                metrics=['mse'])

In [97]:
model.fit(train_ds, validation_data=val_ds, epochs=200)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/200
11/11 [==============================] - 1s 63ms/step - loss: 1224.8395 - mse: 1155.9988 - val_loss: 0.0000e+00 - val_mse: 0.0000e+00
Epoch 2/200
11/11 [==============================] - 0s 5ms/step - loss: 888.5443 - mse: 895.9940 - val_loss: 718.4260 - val_mse: 743.8795
Epoch 3/200
11/11 [==============================] - 0s 4ms/step - loss: 787.3161 - mse: 783.1119 - val_loss: 640.0608 - val_mse: 675.0557
Epoch 4/200
11/11 [==============================] - 0s 4ms/step - loss: 703.2051 - 

In [98]:
loss, mse = model.evaluate(test_ds)
print("Mean Squared Error - Test Data", mse)

4/4 [==============================] - 0s 3ms/step - loss: 54.4169 - mse: 54.7650
Mean Squared Error - Test Data 54.765007
